## Data Transformation and Join
- Author : Will Schultz
- Key Objects : Raw Data, Key Driver Mapping
- Transformations : 
    - Formatted Dates, YYYY-MM-DD
    - Whitespace Removal
    - Null Value Replacement/Solution (Fillna)
    - Remove Special Characters (SQL)  
- Data Type Conversion : Assign data types based on raw columnar data provided  
- Governance : Suffix the column headers with data type identifiers
- Join Requirements : Incorporate 'Key Driver Mapping' data with 'Raw Data' 
- Goal : Create a production-level "Completed" object ready for storage within the data warehouse
- Analysis : Provide answers to business questions

In [1]:
import pandas as pd
import pandasql as ps

In [2]:
raw_data_df = pd.read_csv("raw_data_table.csv")

In [3]:
transformation_query = """SELECT
rd.FeedbackId AS feedback_id,
SUBSTRING(DATE('now'), 1, 2) || SUBSTRING(
    rd.DateCreated,
    INSTR(rd.DateCreated, ' ') - 2,
    2
) || '-' || CASE
    WHEN LENGTH(
        SUBSTRING(
            rd.DateCreated,
            1,
            INSTR(rd.DateCreated, '/') - 1
        )
    ) = 1 THEN '0' || SUBSTRING(
        rd.DateCreated,
        1,
        INSTR(rd.DateCreated, '/') - 1
    )
    ELSE SUBSTRING(
        rd.DateCreated,
        1,
        INSTR(rd.DateCreated, '/') - 1
    )
END || '-' || CASE
    WHEN LENGTH(
        SUBSTRING(
            rd.DateCreated,
            INSTR(rd.DateCreated, '/') + 1,
            INSTR(rd.DateCreated, ' ') - 6
        )
    ) = 1 THEN '0' || SUBSTRING(
        rd.DateCreated,
        INSTR(rd.DateCreated, '/') + 1,
        INSTR(rd.DateCreated, ' ') - 6
    )
    ELSE SUBSTRING(
        rd.DateCreated,
        INSTR(rd.DateCreated, '/') + 1,
        INSTR(rd.DateCreated, ' ') - 6
    )  
END AS date_created_dt,
REPLACE(TRIM(rd.Question), '"', '') question_txt,
CASE WHEN rd.QuestionType = 'Rated0-10' THEN 'rated 0-10'
    WHEN rd.QuestionType = 'Rated1-5' THEN 'rated 1-5'
    WHEN rd.QuestionType = 'YesNo' THEN 'yes no'
    WHEN rd.QuestionType = 'TEXT' THEN 'text'
    WHEN rd.QuestionType = 'FREETEXT' THEN 'free text'
    WHEN rd.QuestionType = 'SINGLESELECT' THEN 'single-select'
    WHEN rd.QuestionType = 'MULTISELECT' THEN 'multi-select'
    ELSE LOWER(rd.QuestionType)
    END AS question_type_txt,
CASE WHEN rd.Answer IS NULL THEN ''
ELSE REPLACE(TRIM(rd.Answer), ';', '') 
END AS answer_txt,
rd.DateFeedbackReceived as date_feedback_received_dt
FROM
    raw_data_df rd"""

transformed_df = ps.sqldf(transformation_query, globals())

display(transformed_df)

transformed_df.dtypes

,feedback_id,date_created_dt,question_txt,question_type_txt,answer_txt,date_feedback_received_dt
0,19548291,2024-02-19,How satisfied are you that the provided techno...,rated 0-10,9,2024-02-19
1,19548291,2024-02-19,Move Support,rated 0-10,9,2024-02-19
2,19548291,2024-02-19,How satisfied are you that your management tea...,rated 0-10,9,2024-02-19
3,19548291,2024-02-19,Change Management/Communications,rated 0-10,8,2024-02-19
4,19548291,2024-02-19,Project Delivery,rated 0-10,9,2024-02-19
...,...,...,...,...,...,...
123,20162642,2024-05-01,"What is the Address, City, State and Country f...",text,,2024-05-01
124,20162642,2024-05-01,Please share your ideas on how we could improv...,text,,2024-05-01
125,20162642,2024-05-01,Do you have specific comments you would like t...,text,,2024-05-01
126,20162642,2024-05-01,Please rate your satisfaction with Enterprise ...,free text,,2024-05-01


feedback_id                   int64
date_created_dt              object
question_txt                 object
question_type_txt            object
answer_txt                   object
date_feedback_received_dt    object
dtype: object

In [4]:
key_driver_mapping_df = pd.read_csv("key_driver_mapping_table.csv")

In [5]:
driver_query = """SELECT
    REPLACE(TRIM(kd.Question), '"', '') AS question_txt,
    kd."Key Driver" AS key_driver_txt
    FROM key_driver_mapping_df kd
    """

driver_df = ps.sqldf(driver_query, globals())

display(driver_df)

,question_txt,key_driver_txt
0,How satisfied are you with the overall deliver...,Delivery
1,How satisfied are you that the provided techno...,Technology
2,How satisfied are you that your management tea...,Change Mgmt/Comms
3,Would you have benefitted from more guidance o...,Change Mgmt/Comms
4,How satisfied are you that the project team me...,Delivery
5,Project Delivery,Delivery
6,Move Support,Move support
7,Change Management/Communications,Change Mgmt/Comms
8,How satisfied are you with the technology solu...,Technology
9,Did you participate in a change management eve...,Change Mgmt/Comms


In [6]:
df = transformed_df.merge(driver_df, how='left', on='question_txt')

final_df = df.fillna('')

display(final_df)

,feedback_id,date_created_dt,question_txt,question_type_txt,answer_txt,date_feedback_received_dt,key_driver_txt
0,19548291,2024-02-19,How satisfied are you that the provided techno...,rated 0-10,9,2024-02-19,Technology
1,19548291,2024-02-19,Move Support,rated 0-10,9,2024-02-19,Move support
2,19548291,2024-02-19,How satisfied are you that your management tea...,rated 0-10,9,2024-02-19,Change Mgmt/Comms
3,19548291,2024-02-19,Change Management/Communications,rated 0-10,8,2024-02-19,Change Mgmt/Comms
4,19548291,2024-02-19,Project Delivery,rated 0-10,9,2024-02-19,Delivery
...,...,...,...,...,...,...,...
123,20162642,2024-05-01,"What is the Address, City, State and Country f...",text,,2024-05-01,
124,20162642,2024-05-01,Please share your ideas on how we could improv...,text,,2024-05-01,
125,20162642,2024-05-01,Do you have specific comments you would like t...,text,,2024-05-01,
126,20162642,2024-05-01,Please rate your satisfaction with Enterprise ...,free text,,2024-05-01,


### Data Transformation Questions

1. Cleanse the data in the 'raw data' and 'key driver mapping' tables.
    - Raw Data = 'transformed_df'
    - Key Driver Mapping Data = 'driver_df'

2. Join the 'Key Driver' value to the raw data.
    - final_df

3. Apply the question: "Please select which workplace location you work at:". The answer should not be blank.
    - final_question_three_df

4. Apply the question: "Please select the role that best describes your involvement in the delivery of your new workplace:". The answer should not be blank.
    - final_question_four_df

5. Adding the 'Answer Score' column and calculating the new rating score based on a '1-5' scale.
    - final_question_five_df

In [7]:
# Analysis query to identify location ids
workplace_query = """SELECT *
    FROM final_df fd
    WHERE fd.question_txt = "Please select which workplace location you work at:"
    """
    
workplace_df = ps.sqldf(workplace_query, globals())

display(workplace_df)

# Adding the workplace location id
answer_txt_data = [
    ['MN101 Twin Cities Consolidation Phase 1', 'MN101'],
    ['TX547', 'TX547'],
    ['CA013 – 7 Technology Dr, Irvine', 'CA013'],
    ['GA019', 'GA019'],
    ['San Antonio, TX (Wellmed)', 'TX547'],
    ['DC007 – 655 New York Avenue', 'DC007']
]
 
question_three_df = pd.DataFrame(answer_txt_data, columns=['answer_txt', 'workplace_location_id'])

# Merging the data frames to create the additional column

merged_question_three_df = final_df.merge(question_three_df, how='left', on='answer_txt')

final_question_three_df = merged_question_three_df.fillna('')

display(final_question_three_df) 

,feedback_id,date_created_dt,question_txt,question_type_txt,answer_txt,date_feedback_received_dt,key_driver_txt,workplace_location_id
0,19548291,2024-02-19,How satisfied are you that the provided techno...,rated 0-10,9,2024-02-19,Technology,
1,19548291,2024-02-19,Move Support,rated 0-10,9,2024-02-19,Move support,
2,19548291,2024-02-19,How satisfied are you that your management tea...,rated 0-10,9,2024-02-19,Change Mgmt/Comms,
3,19548291,2024-02-19,Change Management/Communications,rated 0-10,8,2024-02-19,Change Mgmt/Comms,
4,19548291,2024-02-19,Project Delivery,rated 0-10,9,2024-02-19,Delivery,
...,...,...,...,...,...,...,...,...
123,20162642,2024-05-01,"What is the Address, City, State and Country f...",text,,2024-05-01,,
124,20162642,2024-05-01,Please share your ideas on how we could improv...,text,,2024-05-01,,
125,20162642,2024-05-01,Do you have specific comments you would like t...,text,,2024-05-01,,
126,20162642,2024-05-01,Please rate your satisfaction with Enterprise ...,free text,,2024-05-01,,


In [8]:
# Query to answer the question about your involvement role
role_query = """SELECT *
    FROM final_question_three_df fd
    WHERE fd.question_txt = "Please select the role that best describes your involvement in the delivery of your new workplace:"
    """
    
role_df = ps.sqldf(role_query, globals())

display(role_df)

# Creating the 'Survey Role' column with logic
final_question_three_df['survey_role_txt'] = None 

final_question_three_df['survey_role_txt'] = \
    final_question_three_df['answer_txt'] \
        .where(final_question_three_df['question_txt'] \
            == 'Please select the role that best describes your involvement in the delivery of your new workplace:')
    
final_question_four_df = final_question_three_df.fillna('')

display(final_question_four_df) 

,feedback_id,date_created_dt,question_txt,question_type_txt,answer_txt,date_feedback_received_dt,key_driver_txt,workplace_location_id,survey_role_txt
0,19548291,2024-02-19,How satisfied are you that the provided techno...,rated 0-10,9,2024-02-19,Technology,,
1,19548291,2024-02-19,Move Support,rated 0-10,9,2024-02-19,Move support,,
2,19548291,2024-02-19,How satisfied are you that your management tea...,rated 0-10,9,2024-02-19,Change Mgmt/Comms,,
3,19548291,2024-02-19,Change Management/Communications,rated 0-10,8,2024-02-19,Change Mgmt/Comms,,
4,19548291,2024-02-19,Project Delivery,rated 0-10,9,2024-02-19,Delivery,,
...,...,...,...,...,...,...,...,...,...
123,20162642,2024-05-01,"What is the Address, City, State and Country f...",text,,2024-05-01,,,
124,20162642,2024-05-01,Please share your ideas on how we could improv...,text,,2024-05-01,,,
125,20162642,2024-05-01,Do you have specific comments you would like t...,text,,2024-05-01,,,
126,20162642,2024-05-01,Please rate your satisfaction with Enterprise ...,free text,,2024-05-01,,,


In [9]:
# Creating the 'Answer Score' column based on survey answer
final_question_four_df['answer_score_int'] = None

final_question_four_df['answer_score_int'] = \
    final_question_four_df['answer_txt'] \
        .where(final_question_four_df['question_type_txt'] \
            == 'rated 0-10')

# Casting the column to numeric using Pandas
final_question_five_df = final_question_four_df
final_question_five_df['answer_score_int'] = pd.to_numeric(final_question_five_df['answer_score_int'], downcast='integer', errors='coerce')

# Logic to create the new score
final_question_five_df['answer_score_int'] = \
    final_question_five_df['answer_score_int'] / 2
    
final_question_five_df = final_question_four_df.fillna('')   

display(final_question_five_df) 

,feedback_id,date_created_dt,question_txt,question_type_txt,answer_txt,date_feedback_received_dt,key_driver_txt,workplace_location_id,survey_role_txt,answer_score_int
0,19548291,2024-02-19,How satisfied are you that the provided techno...,rated 0-10,9,2024-02-19,Technology,,,4.5
1,19548291,2024-02-19,Move Support,rated 0-10,9,2024-02-19,Move support,,,4.5
2,19548291,2024-02-19,How satisfied are you that your management tea...,rated 0-10,9,2024-02-19,Change Mgmt/Comms,,,4.5
3,19548291,2024-02-19,Change Management/Communications,rated 0-10,8,2024-02-19,Change Mgmt/Comms,,,4.0
4,19548291,2024-02-19,Project Delivery,rated 0-10,9,2024-02-19,Delivery,,,4.5
...,...,...,...,...,...,...,...,...,...,...
123,20162642,2024-05-01,"What is the Address, City, State and Country f...",text,,2024-05-01,,,,
124,20162642,2024-05-01,Please share your ideas on how we could improv...,text,,2024-05-01,,,,
125,20162642,2024-05-01,Do you have specific comments you would like t...,text,,2024-05-01,,,,
126,20162642,2024-05-01,Please rate your satisfaction with Enterprise ...,free text,,2024-05-01,,,,


In [295]:
# Create a csv to convert to an Excel page
final_question_five_df.to_csv('completed_data.csv', index=False)